In [1]:
from tqdm.notebook import tqdm

from etnn.tools.training_tools import ConfigStore
from etnn.routines.run_config import run_config

In [2]:
config = ConfigStore(
    in_dim=15,
    hidden_dim=128,
    out_dim=1,
    k=2,
    dataset=0,
    ds_size=10_000,
    num_gondolas=10,
    num_part_pg=5,
    loss_name='mse',
    optimizer_name='adam',
    num_max_epochs=100, # old: 20
    learning_rate=0.001,
    batch_size=1024,
    early_stop_tol=5,
    use_equal_batcher=False,
    seed=420,
    label_type="tree",  # tree
    final_label_factor=1/1000
)

In [ ]:
import time

label_list = ["tree", "default"]
k_list = [1, 2, 3, 4, 5]
hidden_list = [8, 32, 128, 512]
counter = 0
total = len(label_list) * len(k_list) * len(hidden_list)

with tqdm(total=total, desc="Progress", unit="run") as pbar:
    for hidden in hidden_list:
        config.hidden_dim = hidden

        for label in label_list:
            config.label_type = label

            for k in k_list:
                config.k = k

                counter += 1

                run_config(
                    config=config,
                    dataset_path="../datasets",
                    results_folder="../results",
                    verbose=False
                )

                pbar.update(1)
                pbar.set_postfix({"hidden": hidden, "label": label, "k": k})

Progress:   0%|          | 0/40 [00:00<?, ?run/s]